In [2]:
import pandas as pd
import numpy as np
import nibabel as nib
import os
import scipy.stats as scp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import itertools
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import statsmodels.formula.api as smf

sns.set(context='talk', style='white', font='Arial')

today = date.today().strftime('%Y%m%d')

#project_dir = '/Users/catcamacho/Library/CloudStorage/Box-Box/CCP/HBN_study/'
project_dir = '/Users/emilyfurtado/Box/HBN_study/'
data_dir = project_dir + 'proc/group/parcel_timeseries/sub_ts/'
out_dir = project_dir + 'stress_analysis/threat_coding/threat_analysis/'
os.makedirs(out_dir,exist_ok=True)

sample_file = project_dir + 'proc/group/datasets_info/sample_gord.32k_fs_LR.pscalar.nii'
atlas_file = project_dir + 'proc/null_lL_WG33/Gordon333_SeitzmanSubcortical.32k_fs_LR.dlabel.nii'
ax0 = nib.load(sample_file).header.get_axis(0)
ax1 = nib.load(sample_file).header.get_axis(1)

TR = 0.8

# load timeseries data info
subinfo = pd.read_csv(project_dir + 'proc/group/datasets/firstleveldatalabels_withpub_thresh0.8_20220412.csv', index_col=0)

# designate stress groups
groups = pd.read_csv(project_dir + 'stress_analysis/factor_analysis/NLES_p_subset_3classes.csv', index_col=0).loc[:,'class'].to_frame()
groups.index = ['sub-{0}'.format(a) for a in groups.index]
groups['class']=['class0{0}'.format(a) for a in groups['class']]

# get network labels
parcel_labels = nib.load(sample_file).header.get_axis(1).name
network_labels = []
for s in parcel_labels:
    b = s.split('_')
    if len(b)<2:
        network_labels.append(b[0])
    else:
        network_labels.append(b[1])
network_labels = np.array(network_labels)
network_names, network_sizes = np.unique(network_labels, return_counts=True)

subinfo = subinfo.drop(['set','cond'], axis=1)
subinfo = subinfo.drop_duplicates()
subinfo = subinfo.merge(groups, how='left', left_index=True, right_index=True)

In [3]:
def compile_ts_data(subdf, movie, datadir, outfile):
    """
    combine data for each movie together into 1 file
    
    Parameters
    ----------
    subdf: DataFrame
        A dataframe with subject IDs as the index. Includes IDs for all usable data.
    movie: str
        Corresponds with the str for the movie content to concatenate (e.g., "DM" or "TP").
    datadir: folder path
        Path to folder with the subject timeseries ciftis.
    outfile: file path
        Path including filename to save the output data of shape Ntimepoints x Nparcels x Nsubjects.
    
    Returns
    -------
    data: numpy array
        The compiled data of shape Ntimepoints x Nparcels x Nsubjects
    """
    if not isinstance(subdf, pd.DataFrame):
        subdf = pd.read_csv(subdf, index_col=0)
    
    for sub in subdf.index:
        file = '{0}{1}_task-movie{2}_bold1_AP_Atlas_rescale_resid0.9_filt_gordonseitzman.32k_fs_LR.ptseries.nii'.format(datadir,sub, movie)
        if sub == subdf.index[0]:
            data = StandardScaler().fit_transform(nib.load(file).get_fdata())
            data = np.expand_dims(data, axis=2)
        else:
            t = StandardScaler().fit_transform(nib.load(file).get_fdata())
            t = np.expand_dims(t, axis=2)
            data = np.concatenate([data,t],axis=2)
    
    print('Compile data from {0} brain regions measured at {1} timepoints from {2} participants.'.format(data.shape[1],data.shape[0],data.shape[2]))
    np.save(outfile, data)
    return(data)

In [4]:
for sample in ['rubic','cbic']:
    for movie in ['TP','DM']:
        sampleinfo = subinfo.loc[(subinfo['site']==sample) & (subinfo['movie']==movie) & ((subinfo['class']=='class01') | (subinfo['class']=='class02') | (subinfo['class']=='class03')),:]
        outdir = os.path.join(out_dir, 'activation', 'pr_nle_groups','dynamic_{0}_movie{1}'.format(sample, movie))
        os.makedirs(outdir, exist_ok=True)
        group_data_file = os.path.join(outdir, 'compiled_timeseries_data_{0}_movie{1}.npy'.format(sample, movie))
        if os.path.isfile(group_data_file):
            group_data = np.load(group_data_file)
        else:
            group_data = compile_ts_data(sampleinfo, movie, data_dir, group_data_file)
            

KeyboardInterrupt: 

In [ ]:
network_names

In [9]:
nois = ['Default', 'VentralAttn', 'CinguloOperc', 'Amygdala']

for movie in ['TP','DM']:
    if movie == 'TP':
        scenes = pd.read_csv(project_dir + "/stress_analysis/threat_coding/movieTP_threat_scenes.csv", index_col = 0)
    elif movie == 'DM':
        scenes = pd.read_csv(project_dir + "/stress_analysis/threat_coding/movieDM_threat_scenes.csv", index_col = 0)
    
    discinfo = subinfo.loc[(subinfo['site']=='rubic') & (subinfo['movie']==movie) & ((subinfo['class']=='class01') | (subinfo['class']=='class02') | (subinfo['class']=='class03')),:]
    repinfo = subinfo.loc[(subinfo['site']=='cbic') & (subinfo['movie']==movie) & ((subinfo['class']=='class01') | (subinfo['class']=='class02') | (subinfo['class']=='class03')),:]    
        
        ####### discovery data #########
    # load data
    disc_ts_file =  os.path.join(out_dir, 'activation', 'pr_nle_groups', 'dynamic_rubic_movie{0}'.format(movie), 
                                 'compiled_timeseries_data_rubic_movie{0}.npy'.format(movie))
    disc_ts = np.load(disc_ts_file)
    # convert to percent signal change
    disc_tss = (disc_ts - disc_ts.min(axis=0, keepdims=True)) / (disc_ts.max(axis=0, keepdims=True) - disc_ts.min(axis=0, keepdims=True))
    mean_sig = np.mean(disc_tss, axis=0, keepdims=True)
    disc_ts = ((disc_tss-mean_sig)/mean_sig)*100

    # average across network/region
    disc_net_ts = np.zeros((disc_ts.shape[0], len(nois), disc_ts.shape[2]))
    for i, n in enumerate(nois):
        disc_net_ts[:,i,:] = np.mean(disc_ts[:,network_labels==n,:], axis=1)

       ####### replication data ######### 
    # load data
    rep_ts_file =  os.path.join(out_dir, 'activation', 'pr_nle_groups', 'dynamic_cbic_movie{0}'.format(movie), 
                                'compiled_timeseries_data_cbic_movie{0}.npy'.format(movie))
    rep_ts = np.load(rep_ts_file)
    # convert to percent signal change
    rep_tss = (rep_ts - rep_ts.min(axis=0, keepdims=True)) / (rep_ts.max(axis=0, keepdims=True) - rep_ts.min(axis=0, keepdims=True))
    mean_sig = np.mean(rep_tss, axis=0, keepdims=True)
    rep_ts = ((rep_tss-mean_sig)/mean_sig)*100

    # average across network/region
    rep_net_ts = np.zeros((rep_ts.shape[0], len(nois), rep_ts.shape[2]))
    for i, n in enumerate(nois):
        rep_net_ts[:,i,:] = np.mean(rep_ts[:,network_labels==n,:], axis=1)
        

    # pull activation for each scene
    for s in scenes.index:
        start = int(scenes.loc[s,'start']/TR)+6
        end = int(scenes.loc[s,'end']/TR)+6
        for i,net in enumerate(nois):
            disc_activation = disc_net_ts[start:end, i, :]
            discinfo['{0}_movie{1}_s{2}'.format(net, movie, s)] = np.mean(disc_activation, axis = 0)
            rep_activation = rep_net_ts[start:end, i, :]
            repinfo['{0}_movie{1}_s{2}'.format(net, movie, s)] = np.mean(rep_activation, axis = 0)
    data = pd.concat([discinfo, repinfo])
    data.to_csv(project_dir + "/stress_analysis/threat_coding/movie{0}_threatactivationdata.csv".format(movie))         
            

/var/folders/0g/g8tyy3vn2zxcv2_bfxpbpr780000gn/T/ipykernel_25149/3020274549.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  discinfo['{0}_movie{1}_s{2}'.format(net, movie, s)] = np.mean(disc_activation, axis = 0)
/var/folders/0g/g8tyy3vn2zxcv2_bfxpbpr780000gn/T/ipykernel_25149/3020274549.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  repinfo['{0}_movie{1}_s{2}'.format(net, movie, s)] = np.mean(rep_activation, axis = 0)


In [10]:
#full data including depression scores
dep_data = pd.read_csv(project_dir+"/stress_analysis/full_data_20220124.csv", index_col = 0)
dep_data = dep_data.loc[:,["MFQ_P_Total", "MFQ_SR_Total", "MDD_mean"]]
dep_data.index.name = "sub"

#neural activation data
data_dm = pd.read_csv(project_dir + "/stress_analysis/threat_coding/movieDM_threatactivationdata.csv", index_col = 0)
data_tp = pd.read_csv(project_dir + "/stress_analysis/threat_coding/movieTP_threatactivationdata.csv", index_col = 0)

# merge activation and full data
data_dm = data_dm.merge(dep_data, how = "left", left_index = True, right_index = True).drop_duplicates()
data_tp = data_tp.merge(dep_data, how = "left", left_index = True, right_index = True).drop_duplicates()

## Despicable Me

In [11]:
#weighted least squares regression model - movie DM (Discovery)
results = pd.DataFrame()
scenes = ["Default_movieDM_s1", "VentralAttn_movieDM_s1", "CinguloOperc_movieDM_s1", "Amygdala_movieDM_s1",
          "Default_movieDM_s3", "VentralAttn_movieDM_s3", "CinguloOperc_movieDM_s3", "Amygdala_movieDM_s3"]
          
data_dm["class_string"] = data_dm["class"]
ind = 0
for site in ["rubic", "cbic"]:
    for x in scenes:
        res = smf.wls("{0} ~ age + class_string + female + meanFD".format(x), 
                  data = data_dm.loc[data_dm['site'] == site,:]).fit()
        results.loc[ind, 'dataset'] = site
        results.loc[ind, 'movie'] = 'DM'
        results.loc[ind, 'sceneNo'] = x
        results.loc[ind, 'group2_tstat'] = res.tvalues['class_string[T.class02]']
        results.loc[ind, 'group2_pval'] = res.pvalues['class_string[T.class02]']
        results.loc[ind, 'group3_tstat'] = res.tvalues['class_string[T.class03]']
        results.loc[ind, 'group3_pval'] = res.pvalues['class_string[T.class03]']
        ind = ind + 1
        print(res.summary())
    
results.to_csv(project_dir + "/stress_analysis/threat_coding/movieDM_threatactivationstressanalysis.csv")

                            WLS Regression Results                            
Dep. Variable:     Default_movieDM_s1   R-squared:                       0.023
Model:                            WLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.323
Date:                Tue, 05 Jul 2022   Prob (F-statistic):              0.254
Time:                        19:00:05   Log-Likelihood:                -822.56
No. Observations:                 286   AIC:                             1657.
Df Residuals:                     280   BIC:                             1679.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 

## The Present

In [12]:
#weighted least squares regression model - movie TP (Discovery)
results = pd.DataFrame()
scenes = ["Default_movieTP_s0", "VentralAttn_movieTP_s0", "CinguloOperc_movieTP_s0", "Amygdala_movieTP_s0"]
          
data_tp["class_string"] = data_tp["class"]
ind = 0
for site in ["rubic", "cbic"]:
    for x in scenes:
        res = smf.wls("{0} ~ age + class_string + female + meanFD".format(x), 
                  data = data_tp.loc[data_tp['site'] == site,:]).fit()
        results.loc[ind, 'dataset'] = site
        results.loc[ind, 'movie'] = 'TP'
        results.loc[ind, 'sceneNo'] = x
        results.loc[ind, 'group2_tstat'] = res.tvalues['class_string[T.class02]']
        results.loc[ind, 'group2_pval'] = res.pvalues['class_string[T.class02]']
        results.loc[ind, 'group3_tstat'] = res.tvalues['class_string[T.class03]']
        results.loc[ind, 'group3_pval'] = res.pvalues['class_string[T.class03]']
        ind = ind + 1
        print(res.summary())
    
results.to_csv(project_dir + "/stress_analysis/threat_coding/movieTP_threatactivationstressanalysis.csv")

                            WLS Regression Results                            
Dep. Variable:     Default_movieTP_s0   R-squared:                       0.007
Model:                            WLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.4704
Date:                Tue, 05 Jul 2022   Prob (F-statistic):              0.798
Time:                        19:00:57   Log-Likelihood:                -943.90
No. Observations:                 329   AIC:                             1900.
Df Residuals:                     323   BIC:                             1923.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                 